<a href="https://colab.research.google.com/github/charlaburnett/change_detection_ukraine/blob/main/change_detection_urkaine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
!cp "/content/drive/MyDrive/Colab Notebooks/credentials.ipynb" .


In [9]:
import import_ipynb
from credentials import config

In [14]:
import ee
ee.Authenticate()
ee.Initialize(project='gee-charlaburnett')

In [16]:
import ee

mariupol = ee.Geometry.Rectangle([37.4, 46.8, 37.7, 47.2])

pre_start = '2021-01-01'
pre_end = '2021-03-01'
post_start = '2022-05-01'
post_end = '2022-07-01'

In [24]:
def get_s1_vv(start, end):
    return (ee.ImageCollection('COPERNICUS/S1_GRD')
            .filterBounds(mariupol)
            .filterDate(start, end)
            .filter(ee.Filter.eq('instrumentMode', 'IW'))
            .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
            .select('VV')
            .mean())


pre_image = get_s1_vv(pre_start, pre_end)
post_image = get_s1_vv(post_start, post_end)


In [28]:
change_image = post_image.subtract(pre_image)

In [33]:
import geemap

Map = geemap.Map(center = [47.1, 37.55], zoom= 11)

vis_params = {
    'min': -2,
    'max': 2,
    'palette': ['blue', 'white', 'red']
}

Map.addLayer(change_image, vis_params, 'SAR Change Detection')
Map.addLayerControl()
Map

Map(center=[47.1, 37.55], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI…

In [40]:
change_red_only = change_image.updateMask(change_image.gt(1.0))
red_vis = {
    'min': 0.5,
    'max': 2,
    'palette': ['red']
}

In [41]:
Map_red = geemap.Map(center=[47.1, 37.55], zoom=11, basemap='SATELLITE')

Map_red.addLayer(change_red_only, red_vis, 'Red-only Positive Change')
Map_red.addLayerControl()
Map_red

Map(center=[47.1, 37.55], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI…

In [42]:
# Remove Watermarks

landcover = ee.Image("ESA/WorldCover/v100/2020")
non_water_mask = landcover.neq(80)

change_red_only_clean = change_red_only.updateMask(non_water_mask)


In [44]:
Map_clean = geemap.Map(center = [47.1, 37.55], zoom= 11, basemap ='Esri.WorldImagery')
Map_clean.addLayer(change_red_only_clean, red_vis, 'Cleaned Red Change')
Map_clean.addLayerControl()
Map_clean

Map(center=[47.1, 37.55], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI…

In [49]:
## Pixel Area Changes

stats = change_red_only_clean.multiply(ee.Image.pixelArea()).reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=mariupol,
    scale = 10,
    maxPixels=1e9
)

area_sq_m = stats.getInfo() ['VV']
area_sq_km = area_sq_m/ 1e6

print(f"Total change area: {area_sq_km:.2f} sq km")


Total change area: 147.20 sq km


In [52]:
#Focus on Urban Areas

landcover = ee.Image("ESA/WorldCover/v100/2020")

urban_mask = landcover.eq(50)

change_urban_only = change_red_only.updateMask(urban_mask)

In [55]:
Map_urban = geemap.Map(center=[47.1, 37.55], zoom=11, basemap='Esri.WorldImagery')

Map_urban.addLayer(change_urban_only, red_vis, 'Urban-only SAR Change')
Map_urban.addLayerControl()
Map_urban


Map(center=[47.1, 37.55], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI…

In [59]:
stats = change_urban_only.multiply(ee.Image.pixelArea()).reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=mariupol,
    scale=10,
    maxPixels=1e9
)

area_sq_m = stats.getInfo()['VV']
area_sq_km = area_sq_m / 1e6
print(f"Urban change area: {area_sq_km:.2f} sq km ")

Urban change area: 43.26 sq km 


In [61]:
total_area_m2 = ee.Image.pixelArea().reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=mariupol,
    scale=10,
    maxPixels=1e9,
).getInfo()['area']

total_area_km2 = total_area_m2 /1e6

In [62]:
percent_changed = (area_sq_km / total_area_km2) * 100
print(f"Urban change area: {area_sq_km:.2f} sq km")
print(f"That's approximately {percent_changed:.1f}% of the AOI ({total_area_km2:.2f} sq km).")

Urban change area: 43.26 sq km
That's approximately 4.3% of the AOI (1014.62 sq km).


In [63]:
#Let's do real Urban boundaries

boundaries = ee.FeatureCollection('WM/geoLab/geoBoundaries/600/ADM2')

In [75]:
ghsl = ee.Image("JRC/GHSL/P2016/BUILT_LDSMT_GLOBE_V1") \
    .select('built') \
    .gt(0)  # Pixels with any built-up value


mariupol_aoi = ee.Geometry.Rectangle([37.55, 47.05, 37.72, 47.17])

mariupol_city_mask = ghsl.updateMask(ghsl).clip(mariupol_aoi)



In [76]:
#SAR-detected change only where GHSL says it's built-up
change_urban_builtup = change_red_only.updateMask(mariupol_city_mask)


In [77]:
# Total changed built-up area
stats = change_urban_builtup.multiply(ee.Image.pixelArea()).reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=mariupol_aoi,
    scale=10,
    maxPixels=1e9
)

area_builtup_km2 = stats.getInfo()['VV'] / 1e6

# Total urban area (from GHSL)
urban_area_stats = mariupol_city_mask.multiply(ee.Image.pixelArea()).reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=mariupol_aoi,
    scale=10,
    maxPixels=1e9
)

total_urban_km2 = urban_area_stats.getInfo()['built'] / 1e6
percent_change = (area_builtup_km2 / total_urban_km2) * 100

print(f"Detected urban SAR change: {area_builtup_km2:.2f} sq km")
print(f"Total built-up city area: {total_urban_km2:.2f} sq km")
print(f"That's {percent_change:.1f}% of Mariupol's built-up area.")


Detected urban SAR change: 64.89 sq km
Total built-up city area: 172.13 sq km
That's 37.7% of Mariupol's built-up area.
